In [2]:
import pandas as pd
import sqlalchemy 
import sql_functions as sf

In [3]:
import matplotlib.pyplot as plt

In [709]:
# load jobs data

schema = 'capstone_datacvpro'

jobs_20 = sf.get_dataframe(f' SELECT * FROM {schema}.analysts_20')
display(jobs_20)

,job_title,salary_estimate,job_description,company_name,location,industry
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation
...,...,...,...,...,...,...
2248,RQS - IHHA - 201900004460 -1q Data Security An...,$78K-$104K (Glassdoor est.),Maintains systems to protect data from unautho...,"Avacend, Inc.\n2.5","Denver, CO",Staffing & Outsourcing
2249,Senior Data Analyst (Corporate Audit),$78K-$104K (Glassdoor est.),Position:\nSenior Data Analyst (Corporate Audi...,Arrow Electronics\n2.9,"Centennial, CO",Wholesale
2250,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),"Title: Technical Business Analyst (SQL, Data a...",Spiceorb,"Denver, CO",-1
2251,"Data Analyst 3, Customer Experience",$78K-$104K (Glassdoor est.),Summary\n\nResponsible for working cross-funct...,Contingent Network Services\n3.1,"Centennial, CO",Enterprise Software & Network Solutions


In [225]:
import re
import nltk
import spacy

In [226]:
# Steps in NLP:

# Tokenizing: split text by word/ sentence 
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

# Stop Words: filter out word that are not needed for context (in, is, and,...)
nltk.download("stopwords")
from nltk.corpus import stopwords

# Stemming: reduce words to their root
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Tagging Parts of Speech: labeling words (noun, pronoun, verb,..)

# Lemmatizing: reduce word to their core meaning
from nltk.stem import WordNetLemmatizer

# Chunking: identify phrases ,word need to be tagged before chunking
from nltk.tokenize import word_tokenize

# Named Entity Recognition: identify named entities, eg. locations, people, organizations,..

[nltk_data] Downloading package punkt to /Users/kgolzev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kgolzev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [227]:
# count words in description column

def count_words(text):
    words = word_tokenize(text)
    return len(words)

jobs_20['desc_words'] = jobs_20['job_description'].apply(count_words)

In [228]:
# count sentences

def count_sent(text):
    sent = sent_tokenize(text)
    return len(sent)

jobs_20['desc_sent'] = jobs_20['job_description'].apply(count_sent)

In [229]:
jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,1162,48
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,709,48
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,802,24
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,888,28
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,573,13


In [230]:
jobs_20.describe()

,desc_words,desc_sent
count,2253.000000,2253.00000
mean,504.766534,17.01154
std,302.406759,13.24425
min,3.000000,1.00000
25%,287.000000,7.00000
50%,462.000000,14.00000
75%,666.000000,23.00000
max,3094.000000,106.00000


In [17]:
# remove stopwords & convert to lower case

def no_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    word_list = []
    for word in words:
        if word.lower() not in stop_words and word.isalpha():   # isalpha checks if characters are alphabetic letters
            word_list.append(word.lower())
    return(word_list)

jobs_20['desc_cleaned'] = jobs_20['job_description'].apply(no_stopwords)

In [12]:
# do not use, gives back high count of words
# remove stopwords and not create list 

def no_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    clean = ' '.join([word for word in words if word not in (stop_words)])
    return(clean)

jobs_20['desc_cleaned'] = jobs_20['job_description'].apply(no_stopwords)

In [18]:
jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent,desc_cleaned,desc_words_clean
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,1162,48,"[eager, roll, sleeves, harness, data, drive, p...",5899
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,709,48,"[overview, provides, analytical, technical, su...",4200
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,802,24,"[looking, senior, data, analyst, love, mentors...",3767
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,888,28,"[requisition, remote, yes, collaborate, create...",4458
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,573,13,"[fanduel, group, fanduel, group, team, brands,...",2771


In [19]:
# count words in cleaned description

def count_words_list(list):
    return len(list)

jobs_20['desc_words_clean'] = jobs_20['desc_cleaned'].apply(count_words_list)

In [20]:
jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent,desc_cleaned,desc_words_clean
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,1162,48,"[eager, roll, sleeves, harness, data, drive, p...",630
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,709,48,"[overview, provides, analytical, technical, su...",422
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,802,24,"[looking, senior, data, analyst, love, mentors...",402
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,888,28,"[requisition, remote, yes, collaborate, create...",459
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,573,13,"[fanduel, group, fanduel, group, team, brands,...",312


In [21]:
jobs_20.describe()

,desc_words,desc_sent,desc_words_clean
count,6162.000000,6162.000000,6162.000000
mean,552.812074,18.711782,308.648815
std,321.742581,13.699206,173.846100
min,3.000000,1.000000,3.000000
25%,320.000000,9.000000,182.000000
50%,510.000000,16.000000,287.000000
75%,726.000000,26.000000,400.000000
max,3420.000000,106.000000,1711.000000


In [295]:
# count occurence of words & show top 5

from collections import Counter

def counter(words):
    return Counter(words).most_common(5)

jobs_20['word_counts'] = jobs_20['desc_cleaned'].apply(counter)

KeyError: 'desc_cleaned'

In [23]:
jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent,desc_cleaned,desc_words_clean,word_counts
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,1162,48,"[eager, roll, sleeves, harness, data, drive, p...",630,"[(data, 19), (justice, 11), (vera, 9), (workin..."
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,709,48,"[overview, provides, analytical, technical, su...",422,"[(data, 18), (quality, 13), (required, 9), (ma..."
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,802,24,"[looking, senior, data, analyst, love, mentors...",402,"[(data, 14), (team, 12), (insights, 8), (custo..."
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,888,28,"[requisition, remote, yes, collaborate, create...",459,"[(business, 11), (celerity, 11), (data, 10), (..."
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,573,13,"[fanduel, group, fanduel, group, team, brands,...",312,"[(fanduel, 10), (team, 6), (reporting, 5), (da..."


In [24]:
# count the most occuring words in cleaned description column

# combine the list in desc_cleaned of each row into one list
all_words = [word for sublist in jobs_20['desc_cleaned'] for word in sublist]

# Count the occurrences of each word
word_counts = Counter(all_words)

# Get the most common tokens
most_common_words = word_counts.most_common(30)

# Print the most common tokens
for word, count in most_common_words:
    print(f'{word}: {count}')

data: 58911
experience: 29776
business: 16561
work: 15186
skills: 11409
team: 11053
analysis: 9084
years: 9051
ability: 8654
development: 8030
management: 7628
knowledge: 7464
analytics: 7064
job: 7053
required: 6958
information: 6944
science: 6900
strong: 6793
support: 6644
working: 6526
including: 6426
solutions: 6336
requirements: 6308
new: 6161
technical: 5965
tools: 5695
research: 5602
systems: 5591
sql: 5251
design: 5240


In [188]:
# check for predefined list of skills

from spacy.matcher import PhraseMatcher

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define a list of skills
skills_list = ["python", "java", "machine learning", "data analysis", "communication", "teamwork"]  # add desired skills

# Initialize phrase matcher
matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp(skill) for skill in skills_list]
matcher.add("SKILLS", None, *patterns)

# Function to extract skills using spaCy and predefined list
def extract_skills_predefined(text):
    doc = nlp(text)
    matches = matcher(doc)
    skills = [doc[start:end].text for match_id, start, end in matches]
    return skills

# Assuming you have a DataFrame named 'jobs_20' with a column 'job_descriptions'
jobs_20['skills_predefined'] = jobs_20['job_description'].apply(extract_skills_predefined)

In [26]:
# use spaCy 

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to extract skills using spaCy
def extract_skills_spacy(text):
    doc = nlp(text)
    skills = []
    for token in doc:
        if token.pos_ == "NOUN" and token.dep_ == "compound":
            skills.append(token.text)
    return skills

jobs_20['skills_spacy'] = jobs_20['job_description'].apply(extract_skills_spacy)

KeyboardInterrupt: 

In [27]:
# extract named entities (people, organizations, locations,..)

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to extract named entities
def extract_named_entities(text):
    doc = nlp(text)
    named_entities = [(ent.text, ent.label_) for ent in doc.ents]
    return named_entities
# Apply the function to the entire DataFrame
jobs_20['named_entities'] = jobs_20['job_description'].apply(extract_named_entities)

In [30]:
# count most occuring entities

# combine the list in desc_cleaned of each row into one list
all_entities = [entity for sublist in jobs_20['named_entities'] for entity in sublist]

# Count the occurrences of each entity
entity_counts = Counter(all_entities)

# Get the most common entities
most_common_entities = entity_counts.most_common(50)

# Print the most common entities
for entity, count in most_common_entities:
    print(f'{entity}: {count}')

('SQL', 'ORG'): 3064
('Tableau', 'GPE'): 1410
('Excel', 'PRODUCT'): 1354
('one', 'CARDINAL'): 1345
('ETL', 'ORG'): 1144
('Bachelor', 'ORG'): 1131
('SAS', 'ORG'): 931
('US', 'GPE'): 894
('Develop', 'ORG'): 795
('Java', 'PERSON'): 712
('5+ years', 'DATE'): 706
('Hadoop', 'ORG'): 703
('PhD', 'WORK_OF_ART'): 679
('2+ years', 'DATE'): 675
('3+ years', 'DATE'): 656
('Microsoft', 'ORG'): 655
('ML', 'ORG'): 640
('U.S.', 'GPE'): 635
('Spark', 'GPE'): 583
('first', 'ORDINAL'): 575
('IBM', 'ORG'): 565
('BI', 'ORG'): 550
('TX', 'ORG'): 548
('AI', 'ORG'): 536
('Identify', 'ORG'): 526
('Strong', 'PERSON'): 490
('the United States', 'GPE'): 466
('SQL', 'PERSON'): 452
('Build', 'GPE'): 444
('Data Science', 'ORG'): 433
('AWS', 'ORG'): 416
('3', 'CARDINAL'): 410
('Python', 'GPE'): 402
('2', 'CARDINAL'): 395
('daily', 'DATE'): 394
('1', 'CARDINAL'): 389
('Job Description', 'PERSON'): 386
('Data Analyst', 'ORG'): 380
('Data Scientist', 'ORG'): 367
('3 years', 'DATE'): 361
('Big Data', 'ORG'): 350
('Chicag

In [226]:
# count unique types of entities

# Initialize a dictionary to count the occurrences of each named entity type
entity_types_count = {}

# Iterate over the named entities found in each row and count the occurrences of each type
for entities in jobs_20['named_entities']:
    for entity in entities:
        entity_type = entity[1]  # Extract the entity type
        if entity_type in entity_types_count:
            entity_types_count[entity_type] += 1
        else:
            entity_types_count[entity_type] = 1
# Display the count of occurrences for each named entity type
for entity_type, count in entity_types_count.items():
    print(f"{entity_type}: {count}")

ORG: 93212
DATE: 17399
CARDINAL: 11660
PRODUCT: 7553
GPE: 20920
PERSON: 19058
WORK_OF_ART: 4420
LOC: 1858
MONEY: 1348
NORP: 3060
LANGUAGE: 381
QUANTITY: 224
PERCENT: 1357
ORDINAL: 1267
LAW: 456
TIME: 981
FAC: 1236
EVENT: 243


In [40]:
# count top required skills for a sanity check

#starting tokenization
skills_list = ["python", "sql", "tableau", "bi tool", "power bi", "aws", "azure", "excel", "powerpoint", "machine learning", "ai", "java", "c++"]  # start with these two skills

# create new columns for each skill
for skill in skills_list:
    jobs_20[skill] = jobs_20['job_description'].apply(lambda x: 1 if skill.lower() in x.lower() else 0)

jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent,desc_cleaned,desc_words_clean,...,bi tool,power bi,aws,azure,excel,powerpoint,machine learning,ai,java,c++
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,1162,48,"[eager, roll, sleeves, harness, data, drive, p...",630,...,0,0,1,0,0,0,0,1,0,0
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,709,48,"[overview, provides, analytical, technical, su...",422,...,0,0,0,0,1,1,0,1,0,0
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,802,24,"[looking, senior, data, analyst, love, mentors...",402,...,1,0,0,0,1,0,0,1,0,0
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,888,28,"[requisition, remote, yes, collaborate, create...",459,...,0,0,1,0,0,0,0,1,0,0
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,573,13,"[fanduel, group, fanduel, group, team, brands,...",312,...,0,0,0,0,1,0,0,1,0,0


In [41]:
# count rows with defined skills

jobs_20[["python", "sql", "tableau", "bi tool", "power bi", "aws", "azure", "excel", "powerpoint", "machine learning", "ai", "java", "c++"]].sum(axis=0).sort_values(ascending=False)

ai                  5492
sql                 3411
excel               3209
python              2694
machine learning    1497
tableau             1293
aws                 1224
java                 996
powerpoint           394
power bi             382
azure                368
c++                  358
bi tool              203
dtype: int64

In [43]:
jobs_20["combined_bi"] = jobs_20["bi tool"] +  jobs_20["power bi"]
jobs_20[["python", "sql", "tableau", "aws", "azure", "excel", "powerpoint", "combined_bi"]].sum(axis=0).sort_values(ascending = False)

sql            3411
excel          3209
python         2694
tableau        1293
aws            1224
combined_bi     585
powerpoint      394
azure           368
dtype: int64